In [1]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
from scipy import stats

In [2]:
# df=pd.read_parquet('test_11_10.parquet')
df = pd.read_csv('Energy_meter_data.csv')
df['Time'] = pd.to_datetime(df['Time'])
df

,Unnamed: 0,ID,MeterId,Time,Phase1_W,Phase1_I,Phase1_V_RY,Phase1_pf,Phase2_W,Phase2_I,...,Phase3_I,Phase3_V_BR,Phase3_pf,W,VA,PF,KWh_Im,KWh_Ex,KVAh,Lh
0,0,200,5,2024-10-11 14:50:20,-50155.187,230.477,422.070,-0.888,-46316.491,215.585,...,229.160,425.986,-0.856,-144816.718,165189.594,-0.876,0.15,10872.050,12385.409,0
1,1,200,6,2024-10-11 14:50:23,0.000,0.000,0.077,1.000,0.000,0.000,...,0.000,0.159,1.000,0.000,0.000,1.000,0.20,12178.529,13968.280,0
2,2,200,1,2024-10-11 14:51:25,-41153.499,196.609,423.652,-0.852,43847.551,206.069,...,222.923,426.531,-0.885,-45471.722,52674.738,-0.862,0.17,3827.970,4434.330,0
3,3,200,2,2024-10-11 14:51:28,-49110.960,224.704,421.696,-0.892,-44273.101,205.183,...,218.013,426.399,-0.859,-139557.952,158421.938,-0.880,0.17,12353.609,14062.130,0
4,4,200,3,2024-10-11 14:51:31,0.000,0.000,0.066,1.000,0.000,0.000,...,0.000,0.117,1.000,0.000,0.000,1.000,0.17,17370.529,19702.898,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15025,15025,200,3,2024-10-14 05:03:41,-46198.144,224.134,419.735,-0.849,-45633.644,209.988,...,233.056,424.852,-0.893,-143259.687,162178.141,-0.882,0.18,25457.359,28873.279,0
15026,15026,200,5,2024-10-14 05:03:44,0.000,0.000,0.000,1.000,0.000,0.000,...,0.000,0.000,1.000,0.000,0.000,1.000,0.15,17638.180,20094.420,0
15027,15027,200,6,2024-10-14 05:03:47,-47805.085,220.735,419.352,-0.887,-42786.487,199.529,...,214.282,425.131,-0.848,-135206.546,154421.516,-0.875,0.20,19675.951,22573.859,0
15028,15028,200,1,2024-10-14 05:04:49,-40487.886,193.138,426.619,-0.843,43316.621,201.048,...,222.053,431.811,-0.881,-45564.815,53693.805,-0.848,0.17,4334.320,5022.270,0


In [3]:
check = ['Phase1_I', 'Phase2_I', 'Phase3_I', 'Phase1_V_RY', 'Phase2_V_YB', 'Phase3_V_BR']
df['Status'] = (df[check] >= 10).all(axis=1)

In [4]:
df.rename(columns={'Phase1_W':'Phase1_kW','Phase2_W':'Phase_kW', 'Phase3_W':'Phase3_kW', 'W':'kW'})
list1 = ['Phase1_W', 'Phase2_W', 'Phase3_W', 'W','VA']
list2 =['Phase1_W','Phase2_W','Phase3_W','W','PF']
list_negative = ['Phase1_pf','Phase2_pf','Phase3_pf','Phase1_W','Phase2_W','Phase3_W','PF']
df[list1] = df[list1].apply(lambda x: x / 1000)
df[list2] = df[list2].apply(lambda x: x *-1)
#df[list_pf] = df[list_pf].apply(lambda x: x.where(x < 0, -1*x))
#df[list_negative] = df[list_negative].apply(lambda x: x.where(x >= 0, -x))
df[list_negative] = df[list_negative].apply(lambda x: x.where(x >= 0.000000, abs(x)))

In [5]:
def calculate_voltage_imbalance(row):
    ##### Pump considered off when Phase current and volatge = 0
    currents_volt = [row['Phase1_I'], row['Phase2_I'], row['Phase3_I'],row['Phase1_V_RY'], row['Phase2_V_YB'], row['Phase3_V_BR']]
    if all(i != 0 for i in currents_volt): 
        voltages = [row['Phase1_V_RY'], row['Phase2_V_YB'], row['Phase3_V_BR']]
  #  if all(v != 0 for v in voltages):  # not applicable hen the Pump is off
        avg_voltage = sum(voltages) / len(voltages)
        max_deviation = max(voltages) - avg_voltage
        voltage_imbalance = (max_deviation / avg_voltage) * 100
        return voltage_imbalance
    else:
        return None


In [6]:
def calculate_current_imbalance(row):
    currents = [row['Phase1_I'], row['Phase2_I'], row['Phase3_I']]
    if all(i != 0 for i in currents):  # doesnot apply when the pump is off
        avg_current = sum(currents) / len(currents)
        max_current_deviation = max(currents)-avg_current
        current_imbalance = ((max_current_deviation) / avg_current) * 100
        return current_imbalance
    else:
        return None

In [7]:
df['Voltage_Imbalance_%'] = df.apply(calculate_voltage_imbalance, axis=1)
df['Current_Imbalance_%'] = df.apply(calculate_current_imbalance, axis=1)
df

,Unnamed: 0,ID,MeterId,Time,Phase1_W,Phase1_I,Phase1_V_RY,Phase1_pf,Phase2_W,Phase2_I,...,W,VA,PF,KWh_Im,KWh_Ex,KVAh,Lh,Status,Voltage_Imbalance_%,Current_Imbalance_%
0,0,200,5,2024-10-11 14:50:20,50.155187,230.477,422.070,0.888,46.316491,215.585,...,144.816718,165.189594,0.876,0.15,10872.050,12385.409,0,True,0.501502,2.400544
1,1,200,6,2024-10-11 14:50:23,-0.000000,0.000,0.077,1.000,-0.000000,0.000,...,-0.000000,0.000000,1.000,0.20,12178.529,13968.280,0,False,NaN,NaN
2,2,200,1,2024-10-11 14:51:25,41.153499,196.609,423.652,0.852,43.847551,206.069,...,45.471722,52.674738,0.862,0.17,3827.970,4434.330,0,True,0.600256,6.900245
3,3,200,2,2024-10-11 14:51:28,49.110960,224.704,421.696,0.892,44.273101,205.183,...,139.557952,158.421938,0.880,0.17,12353.609,14062.130,0,True,0.602816,4.045686
4,4,200,3,2024-10-11 14:51:31,-0.000000,0.000,0.066,1.000,-0.000000,0.000,...,-0.000000,0.000000,1.000,0.17,17370.529,19702.898,0,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15025,15025,200,3,2024-10-14 05:03:41,46.198144,224.134,419.735,0.849,45.633644,209.988,...,143.259687,162.178141,0.882,0.18,25457.359,28873.279,0,True,0.834648,4.794822
15026,15026,200,5,2024-10-14 05:03:44,-0.000000,0.000,0.000,1.000,-0.000000,0.000,...,-0.000000,0.000000,1.000,0.15,17638.180,20094.420,0,False,NaN,NaN
15027,15027,200,6,2024-10-14 05:03:47,47.805085,220.735,419.352,0.887,42.786487,199.529,...,135.206546,154.421516,0.875,0.20,19675.951,22573.859,0,True,0.791224,4.358864
15028,15028,200,1,2024-10-14 05:04:49,40.487886,193.138,426.619,0.843,43.316621,201.048,...,45.564815,53.693805,0.848,0.17,4334.320,5022.270,0,True,0.887278,8.100753


In [8]:
df.set_index('Time', inplace=True)

In [9]:
df.dtypes

Unnamed: 0               int64
ID                       int64
MeterId                  int64
Phase1_W               float64
Phase1_I               float64
Phase1_V_RY            float64
Phase1_pf              float64
Phase2_W               float64
Phase2_I               float64
Phase2_V_YB            float64
Phase2_pf              float64
Phase3_W               float64
Phase3_I               float64
Phase3_V_BR            float64
Phase3_pf              float64
W                      float64
VA                     float64
PF                     float64
KWh_Im                 float64
KWh_Ex                 float64
KVAh                   float64
Lh                       int64
Status                    bool
Voltage_Imbalance_%    float64
Current_Imbalance_%    float64
dtype: object

In [10]:
df[['MeterId','W']]

,MeterId,W
Time,,
2024-10-11 14:50:20,5,144.816718
2024-10-11 14:50:23,6,-0.000000
2024-10-11 14:51:25,1,45.471722
2024-10-11 14:51:28,2,139.557952
2024-10-11 14:51:31,3,-0.000000
...,...,...
2024-10-14 05:03:41,3,143.259687
2024-10-14 05:03:44,5,-0.000000
2024-10-14 05:03:47,6,135.206546


In [11]:
def meter_id_correction(row):
    if row['MeterId'] == 1:
        return row['Phase1_W']+row['Phase2_W']+row['Phase3_W']
    else:
        return row['W']

In [12]:
df['W'] = df.apply(meter_id_correction,axis=1)

In [13]:
df[['MeterId','W']]

,MeterId,W
Time,,
2024-10-11 14:50:20,5,144.816718
2024-10-11 14:50:23,6,-0.000000
2024-10-11 14:51:25,1,133.166920
2024-10-11 14:51:28,2,139.557952
2024-10-11 14:51:31,3,-0.000000
...,...,...
2024-10-14 05:03:41,3,143.259687
2024-10-14 05:03:44,5,-0.000000
2024-10-14 05:03:47,6,135.206546


In [14]:
head= 39 ## m
flow_rate = 932/3600 ## m3/s
density = 1000 ### kg/m3
g = 9.8 ## m/s2

P_out= (density*g*flow_rate*head)/1000    # power in kW

In [15]:
df['Efficiency'] = (P_out/df['W'])*100
df

,Unnamed: 0,ID,MeterId,Phase1_W,Phase1_I,Phase1_V_RY,Phase1_pf,Phase2_W,Phase2_I,Phase2_V_YB,...,VA,PF,KWh_Im,KWh_Ex,KVAh,Lh,Status,Voltage_Imbalance_%,Current_Imbalance_%,Efficiency
Time,,,,,,,,,,,,,,,,,,,,,
2024-10-11 14:50:20,0,200,5,50.155187,230.477,422.070,0.888,46.316491,215.585,423.525,...,165.189594,0.876,0.15,10872.050,12385.409,0,True,0.501502,2.400544,68.325905
2024-10-11 14:50:23,1,200,6,-0.000000,0.000,0.077,1.000,-0.000000,0.000,0.077,...,0.000000,1.000,0.20,12178.529,13968.280,0,False,NaN,NaN,-inf
2024-10-11 14:51:25,2,200,1,41.153499,196.609,423.652,0.852,43.847551,206.069,421.775,...,52.674738,0.862,0.17,3827.970,4434.330,0,True,0.600256,6.900245,74.303238
2024-10-11 14:51:28,3,200,2,49.110960,224.704,421.696,0.892,44.273101,205.183,423.437,...,158.421938,0.880,0.17,12353.609,14062.130,0,True,0.602816,4.045686,70.900534
2024-10-11 14:51:31,4,200,3,-0.000000,0.000,0.066,1.000,-0.000000,0.000,0.063,...,0.000000,1.000,0.17,17370.529,19702.898,0,False,NaN,NaN,-inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-14 05:03:41,15025,200,3,46.198144,224.134,419.735,0.849,45.633644,209.988,419.419,...,162.178141,0.882,0.18,25457.359,28873.279,0,True,0.834648,4.794822,69.068511
2024-10-14 05:03:44,15026,200,5,-0.000000,0.000,0.000,1.000,-0.000000,0.000,0.000,...,0.000000,1.000,0.15,17638.180,20094.420,0,False,NaN,NaN,-inf
2024-10-14 05:03:47,15027,200,6,47.805085,220.735,419.352,0.887,42.786487,199.529,420.898,...,154.421516,0.875,0.20,19675.951,22573.859,0,True,0.791224,4.358864,73.182354


In [16]:
df = df[df['Status']!=False]

In [17]:
df.shape

(8038, 26)

In [18]:
df.isna().sum()

Unnamed: 0             0
ID                     0
MeterId                0
Phase1_W               0
Phase1_I               0
Phase1_V_RY            0
Phase1_pf              0
Phase2_W               0
Phase2_I               0
Phase2_V_YB            0
Phase2_pf              0
Phase3_W               0
Phase3_I               0
Phase3_V_BR            0
Phase3_pf              0
W                      0
VA                     0
PF                     0
KWh_Im                 0
KWh_Ex                 0
KVAh                   0
Lh                     0
Status                 0
Voltage_Imbalance_%    0
Current_Imbalance_%    0
Efficiency             0
dtype: int64

In [19]:
df[df['Voltage_Imbalance_%'].isna()]

,Unnamed: 0,ID,MeterId,Phase1_W,Phase1_I,Phase1_V_RY,Phase1_pf,Phase2_W,Phase2_I,Phase2_V_YB,...,VA,PF,KWh_Im,KWh_Ex,KVAh,Lh,Status,Voltage_Imbalance_%,Current_Imbalance_%,Efficiency
Time,,,,,,,,,,,,,,,,,,,,,


In [20]:
df.iloc[:,:19]

,Unnamed: 0,ID,MeterId,Phase1_W,Phase1_I,Phase1_V_RY,Phase1_pf,Phase2_W,Phase2_I,Phase2_V_YB,Phase2_pf,Phase3_W,Phase3_I,Phase3_V_BR,Phase3_pf,W,VA,PF,KWh_Im
Time,,,,,,,,,,,,,,,,,,,
2024-10-11 14:50:20,0,200,5,50.155187,230.477,422.070,0.888,46.316491,215.585,423.525,0.883,48.344929,229.160,425.986,0.856,144.816718,165.189594,0.876,0.15
2024-10-11 14:51:25,2,200,1,41.153499,196.609,423.652,0.852,43.847551,206.069,421.775,0.879,48.165870,222.923,426.531,0.885,133.166920,52.674738,0.862,0.17
2024-10-11 14:51:28,3,200,2,49.110960,224.704,421.696,0.892,44.273101,205.183,423.437,0.887,46.148925,218.013,426.399,0.859,139.557952,158.421938,0.880,0.17
2024-10-11 14:51:33,5,200,5,50.646601,232.442,421.462,0.889,46.068210,214.229,422.994,0.886,48.221726,229.527,426.128,0.853,145.064093,165.377766,0.876,0.15
2024-10-11 14:52:39,7,200,1,41.043964,198.210,421.439,0.857,43.426156,202.807,419.000,0.887,48.224796,221.673,424.509,0.893,132.694916,52.763867,0.868,0.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-14 05:02:34,15022,200,6,47.901347,218.026,426.603,0.886,42.490772,196.271,428.672,0.883,44.889495,212.981,432.655,0.843,135.291515,155.142500,0.871,0.20
2024-10-14 05:03:36,15023,200,1,40.511948,192.752,420.350,0.852,43.778820,203.160,420.058,0.889,48.388800,223.704,425.404,0.884,132.679568,52.946492,0.851,0.17
2024-10-14 05:03:41,15025,200,3,46.198144,224.134,419.735,0.849,45.633644,209.988,419.419,0.903,51.427983,233.056,424.852,0.893,143.259687,162.178141,0.882,0.18


In [21]:
df.columns

Index(['Unnamed: 0', 'ID', 'MeterId', 'Phase1_W', 'Phase1_I', 'Phase1_V_RY',
       'Phase1_pf', 'Phase2_W', 'Phase2_I', 'Phase2_V_YB', 'Phase2_pf',
       'Phase3_W', 'Phase3_I', 'Phase3_V_BR', 'Phase3_pf', 'W', 'VA', 'PF',
       'KWh_Im', 'KWh_Ex', 'KVAh', 'Lh', 'Status', 'Voltage_Imbalance_%',
       'Current_Imbalance_%', 'Efficiency'],
      dtype='object')

In [80]:
df[df.index>='2024-10-13']

,Unnamed: 0,ID,MeterId,Phase1_W,Phase1_I,Phase1_V_RY,Phase1_pf,Phase2_W,Phase2_I,Phase2_V_YB,...,VA,PF,KWh_Im,KWh_Ex,KVAh,Lh,Status,Voltage_Imbalance_%,Current_Imbalance_%,Efficiency
Time,,,,,,,,,,,,,,,,,,,,,
2024-10-13 00:00:23,7990,200,3,45.592151,219.194,425.633,0.846,46.455140,209.414,426.075,...,163.219391,0.881,0.17,21985.619,24932.881,0,True,0.783573,5.739184,68.709333
2024-10-13 00:00:25,7991,200,5,50.721269,231.111,425.952,0.886,46.457022,213.111,425.541,...,164.746703,0.878,0.15,15081.750,17182.471,0,True,0.784003,3.787238,68.361066
2024-10-13 00:00:28,7992,200,6,47.781616,219.982,425.826,0.877,43.737226,201.182,425.165,...,155.318578,0.872,0.20,16538.230,18967.969,0,True,0.819123,4.678729,73.007589
2024-10-13 00:01:36,7995,200,3,45.731194,219.799,425.901,0.845,46.118519,207.797,425.657,...,162.924656,0.883,0.17,21988.541,24936.189,0,True,0.794584,5.880243,68.731452
2024-10-13 00:01:38,7996,200,5,50.810800,231.010,425.596,0.889,46.006312,211.153,425.826,...,164.375531,0.878,0.15,15084.689,17185.820,0,True,0.794400,3.963649,68.519904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-14 05:02:34,15022,200,6,47.901347,218.026,426.603,0.886,42.490772,196.271,428.672,...,155.142500,0.871,0.20,19673.199,22570.711,0,True,0.779157,4.272428,73.136392
2024-10-14 05:03:36,15023,200,1,40.511948,192.752,420.350,0.852,43.778820,203.160,420.058,...,52.946492,0.851,0.17,4333.380,5021.180,0,True,0.821607,8.310954,74.576165
2024-10-14 05:03:41,15025,200,3,46.198144,224.134,419.735,0.849,45.633644,209.988,419.419,...,162.178141,0.882,0.18,25457.359,28873.279,0,True,0.834648,4.794822,69.068511


In [22]:
meter_dfs = {}
for i in df['MeterId'].unique():
    meter_dfs[f'Pump_{i}'] = df[df['MeterId'] == i][[#'Time',
         'Phase1_W', 'Phase1_I', 'Phase1_V_RY', 'Phase1_pf',
        'Phase2_W', 'Phase2_I', 'Phase2_V_YB', 'Phase2_pf',
        'Phase3_W', 'Phase3_I', 'Phase3_V_BR', 'Phase3_pf',
        'W', 'VA', 'PF', 'KWh_Im', 'KWh_Ex', 'KVAh', 'Lh','Status','Voltage_Imbalance_%','Current_Imbalance_%',
        'Efficiency'
    ]]

In [23]:
pump_1_df = meter_dfs['Pump_1']
pump_2_df = meter_dfs['Pump_2']
pump_3_df = meter_dfs['Pump_3']
pump_5_df = meter_dfs['Pump_5']
pump_6_df = meter_dfs['Pump_6']

In [24]:
df1= pump_1_df[pump_1_df['Status']!=False]
df2= pump_2_df[pump_2_df['Status']!=False]
df3= pump_3_df[pump_3_df['Status']!=False]
df5= pump_5_df[pump_5_df['Status']!=False]
df6= pump_6_df[pump_6_df['Status']!=False]

In [25]:
df1.columns

Index(['Phase1_W', 'Phase1_I', 'Phase1_V_RY', 'Phase1_pf', 'Phase2_W',
       'Phase2_I', 'Phase2_V_YB', 'Phase2_pf', 'Phase3_W', 'Phase3_I',
       'Phase3_V_BR', 'Phase3_pf', 'W', 'VA', 'PF', 'KWh_Im', 'KWh_Ex', 'KVAh',
       'Lh', 'Status', 'Voltage_Imbalance_%', 'Current_Imbalance_%',
       'Efficiency'],
      dtype='object')

In [26]:
# df1['Efficiency'] = (P_out/df1['W'])*100
# df2['Efficiency'] = (P_out/df2['W'])*100
# df3['Efficiency'] = (P_out/df3['W'])*100
# df5['Efficiency'] = (P_out/df5['W'])*100
# df6['Efficiency'] = (P_out/df6['W'])*100

In [77]:
fig = go.Figure()
# dfs = [df1,df1,df1,df1,df1]
# pump = ''
# for i in dfs:
#     if i == df1:
#         pump = 'Pump 1'
#     elif i == df1:
#         pump = 'Pump 2'
#     elif i == df1:
#         pump = 'Pump 3'
#     elif i == df1:
#         pump = 'Pump 5'
#     else:
#         pump = 'Pump 6'
# i = df1
fig.add_trace(go.Scatter(
    x=df1.index,
    y=df1['Voltage_Imbalance_%'],
    mode='lines+markers',
    name='Voltage Imbalance_%',
    line=dict(color='blue'),
    marker=dict(size=5)
))

fig.add_trace(go.Scatter(
    x=df1.index,
    y=df1['Current_Imbalance_%'],
    mode='lines+markers',
    name='Current Imbalance_%',
    line=dict(color='red'),
    marker=dict(size=5)
))

fig.update_layout(
    title=f'Variation of Current and Voltage Imbalances for Pump 1 Over Time',
    xaxis_title='Time',
    yaxis_title='Imbalance %',
    legend_title='Legend',
    template='plotly_white'
)
fig.update_yaxes(
    range=[0, 12], 
    dtick=1         
)


fig.show()

In [28]:
# fig = go.Figure()

# fig.add_trace(go.Scatter(
#     x=df5.index,
#     y=df5['Phase1_V_RY'],
#     mode='lines+markers',
#     name='Voltage RY',
#     line=dict(color='red'),
#     marker=dict(size=5)
# ))


# # fig.add_trace(go.Scatter(
# #     x=df5['Time'],
# #     y=df5['Phase2_V_YB'],
# #     mode='lines+markers',
# #     name='Current Imbalance',
# #     line=dict(color='yellow'),
# #     marker=dict(size=5)
# # ))
# # fig.add_trace(go.Scatter(
# #     x=df5['Time'],
# #     y=df5['Phase3_V_BR'],
# #     mode='lines+markers',
# #     name='Current Imbalance',
# #     line=dict(color='blue'),
# #     marker=dict(size=5)
# # ))


# fig.update_layout(
#     title='Variation of Current and Voltage Imbalances Over Time',
#     xaxis_title='Time',
#     yaxis_title='Imbalance',
#     legend_title='Legend',
#     template='plotly_white'
# )

# fig.show()

In [54]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df1.index,
    y=df1['PF'],
    mode='lines+markers',
#     line_shape='spline',
    name='Voltage RY',
    line=dict(color='red'),
    marker=dict(size=5)
))
fig.update_layout(
    title='Variation of PF for Pump 1 Over Time',
    xaxis_title='Time',
    yaxis_title='Power Factor',
    legend_title='Legend',
    template='plotly_white'
)
fig.update_yaxes(
    range=[0.6, 1], 
    dtick=0.02         
)


fig.show()

In [30]:
# fig = go.Figure()

# # Add a trace for Voltage Imbalance
# fig.add_trace(go.Scatter(
#     x=df6.index,
#     y=df6['W'],
#     mode='lines+markers',
#     name='Voltage RY',
#     line=dict(color='red'),
#     marker=dict(size=5)
# ))
# fig.update_layout(
#     title='Variation of Power of Pump6 Over Time',
#     xaxis_title='Time',
#     yaxis_title='Imbalance',
#     legend_title='Legend',
#     template='plotly_white'
# )
# fig.update_yaxes(
#     range=[120, 150]
# #     dtick=0.02         
# )


# fig.show()

In [60]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df6.index,
    y=df6['Efficiency'],
    mode='lines+markers',
 #   name='Voltage Imbalance_%',
    line=dict(color='blue'),
    marker=dict(size=5)
))


fig.update_layout(
    title='Variation of Efficiency for Pump 6 Over Time',
    xaxis_title='Time',
    yaxis_title='Efficiency %',
    legend_title='Legend',
    template='plotly_white'
)
fig.update_yaxes(
    range=[60, 100],  
    dtick=10         
)

fig.show()

In [65]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df1.index,
    y=df1['W'],
    mode='lines+markers',
    name='Voltage Imbalance_%',
    line=dict(color='blue'),
    marker=dict(size=5)
))


fig.add_shape(
    type='line',
    x0=df1.index.min(),  
    y0=132,              ## Rated power of motor
    x1=df1.index.max(),  
    y1=132,               # Y-value for the reference line
    line=dict(color='green', width=2, dash='dash'),  # Line style
)


fig.update_layout(
    title='Variation of Power for Pump 1 Over Time',
    xaxis_title='Time',
    yaxis_title='Power (kW)',
    legend_title='Legend',
    template='plotly_white'
)
fig.update_yaxes(
    range=[120, 150]
#     dtick=0.02         
)

fig.show()

In [32]:
# fig = go.Figure()

# fig.add_trace(go.Scatter(
#     x=df1.index,
#     y=df1['W'],
#     mode='lines+markers',
#     name='Voltage Imbalance_%',
#     line=dict(color='blue'),
#     marker=dict(size=5)
# ))


# fig.add_shape(
#     type='line',
#     x0=df1.index.min(),  
#     y0=132,               ## rated power-motor
#     x1=df1.index.max(),  
#     y1=132,               
#     line=dict(color='green', width=2, dash='dash'),  # Line style
# )


# fig.update_layout(
#     title='Power Variation of Pump1 Over Time',
#     xaxis_title='Time',
#     yaxis_title='Imbalance',
#     legend_title='Legend',
#     template='plotly_white'
# )
# fig.update_yaxes(
#     range=[120, 150],  
#     dtick=10         
# )

# fig.show()

In [34]:
# fig = go.Figure()

# fig.add_trace(go.Scatter(
#     x=df5.index,
#     y=df5['Efficiency'],
#     mode='lines+markers',
#     name='Voltage Imbalance_%',
#     line=dict(color='blue'),
#     marker=dict(size=5)
# ))


# fig.update_layout(
#     title='Efficiency Variation of Pump5',
#     xaxis_title='Time',
#     yaxis_title='Efficiency %',
#     legend_title='Legend',
#     template='plotly_white'
# )
# fig.update_yaxes(
#     range=[60, 80],  
#     dtick=2         
# )

# fig.show()

In [36]:
# fig = go.Figure()

# fig.add_trace(go.Scatter(
#     x=df3.index,
#     y=df3['Efficiency'],
#     mode='lines+markers',
#     name='Voltage Imbalance_%',
#     line=dict(color='blue'),
#     marker=dict(size=5)
# ))


# fig.update_layout(
#     title='Efficiency Variation of Pump3',
#     xaxis_title='Time',
#     yaxis_title='Efficiency %',
#     legend_title='Legend',
#     template='plotly_white'
# )
# fig.update_yaxes(
#     range=[60, 80],  
#     dtick=10         
# )

# fig.show()

In [44]:
# fig = go.Figure()

# fig.add_trace(go.Scatter(
#     x=df6.index,
#     y=df6['Efficiency'],
#     mode='lines+markers',
#     name='Voltage Imbalance_%',
#     line=dict(color='blue'),
#     marker=dict(size=5)
# ))


# fig.update_layout(
#     title='Efficiency Variation of Pump6',
#     xaxis_title='Time',
#     yaxis_title='Efficiency %',
#     legend_title='Legend',
#     template='plotly_white'
# )
# fig.update_yaxes(
#     range=[60, 90],  
#     dtick=10         
# )

# fig.show()